In [1]:
import cv2
import numpy as np
import qrcode
from PIL import Image
from pyzbar.pyzbar import decode

1. provide image path
2. provide qr code data. Can be a `str` or `dict`
3. provide path for embedded image

*note: the image must be at least 512 x 512 in size. this is due to the limits of the coefficients after inverting the image, which is an inevitably a lossy process and therefor, requires a larger qr code to be generated. This setting can be changed by altering the `qr_code_size` parameter within the `generate_qr_code` function, however, in my limited 

In [2]:
image = '/Users/wm/Code/projects/watermark/images/original_images/beach.jpeg'
qr_code_data = 'https://example.com'

Crops or pads the image to ensure its dimensions are divisible by 8.

In [3]:
def crop_or_pad_image_to_divisible_by_8(image):

    height, width, _ = image.shape
    new_height = (height // 8) * 8
    new_width = (width // 8) * 8
    return image[:new_height, :new_width]

In [4]:
original_image = cv2.imread(image)
original_image = crop_or_pad_image_to_divisible_by_8(original_image)

Generates a tiled QR code image that covers the entire image size.

In [20]:
def generate_tiled_qr_code(data, image_width, image_height, qr_code_size=512):
    
    qr = qrcode.QRCode(error_correction=qrcode.constants.ERROR_CORRECT_H)
    qr.add_data(data)
    qr.make(fit=True)
    qr_image = qr.make_image(fill='black', back_color='white').convert('L')
    qr_image = qr_image.resize((qr_code_size, qr_code_size))
    
    # Create a blank canvas to hold the tiled QR codes
    tiled_qr_image = Image.new('L', (image_width, image_height), color=255)
    
    # Tile the QR code across the canvas
    num_qr_x = image_width // qr_code_size
    num_qr_y = image_height // qr_code_size
    
    for i in range(num_qr_y):
        for j in range(num_qr_x):
            x_offset = j * qr_code_size
            y_offset = i * qr_code_size
            tiled_qr_image.paste(qr_image, (x_offset, y_offset))

    tiled_qr_image.save('tiled_qr_image.png')
    
    return tiled_qr_image

In [6]:
image_height, image_width, _ = original_image.shape
tiled_qr_image = generate_tiled_qr_code(qr_code_data, image_width, image_height)

Applies DCT to the Y channel of the image (in YCrCb color space).

In [7]:
def apply_dct(image):
    
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    
    dct_blocks = np.zeros_like(y, dtype=np.float32)
    height, width = y.shape
    
    # Apply DCT block-wise
    for i in range(0, height, 8):
        for j in range(0, width, 8):
            block = y[i:i+8, j:j+8]
            dct_blocks[i:i+8, j:j+8] = cv2.dct(np.float32(block))
    
    return dct_blocks, ycrcb

In [8]:
dct_blocks, ycrcb = apply_dct(original_image)

Embeds a tiled QR code image into the DCT coefficients of the Y channel.

In [9]:
def embed_tiled_qr_in_dct(dct_blocks, tiled_qr_image, embed_strength=25.0):
    
    qr_data = np.array(tiled_qr_image)
    qr_data = np.where(qr_data > 128, 1, 0)
    
    height, width = dct_blocks.shape
    
    for i in range(0, height, 8):
        for j in range(0, width, 8):
            qr_bit = qr_data[i, j]
            block_dct = dct_blocks[i:i+8, j:j+8]
            
            # Embed QR code bits into DCT high-frequency coefficients
            if qr_bit == 1:
                block_dct[5, 5] += embed_strength * 0.8
                block_dct[6, 6] += embed_strength * 1.0
                block_dct[7, 7] += embed_strength * 0.5
            else:
                block_dct[5, 5] -= embed_strength * 0.8
                block_dct[6, 6] -= embed_strength * 1.0
                block_dct[7, 7] -= embed_strength * 0.5
    
    return dct_blocks

In [10]:
dct_blocks = embed_tiled_qr_in_dct(dct_blocks, tiled_qr_image)

Applies Inverse DCT and reconstructs the image in the spatial domain.

In [11]:
def inverse_dct_and_save(dct_blocks, ycrcb, output_filename='watermarked_image.jpg'):

    y, cr, cb = cv2.split(ycrcb)
    height, width = dct_blocks.shape

    for i in range(0, height, 8):
        for j in range(0, width, 8):
            block = dct_blocks[i:i+8, j:j+8]
            y[i:i+8, j:j+8] = cv2.idct(block)
    
    watermarked_image = cv2.cvtColor(cv2.merge([y, cr, cb]), cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(output_filename, watermarked_image)
    return watermarked_image

In [12]:
watermarked_image = inverse_dct_and_save(dct_blocks, ycrcb)

Compare the Y channels (luminance) of the original and watermarked images.

In [13]:
def compare_y_channels(original_image, watermarked_image):

    ycrcb_original = cv2.cvtColor(original_image, cv2.COLOR_BGR2YCrCb)
    ycrcb_watermarked = cv2.cvtColor(watermarked_image, cv2.COLOR_BGR2YCrCb)
    
    y_original, _, _ = cv2.split(ycrcb_original)
    y_watermarked, _, _ = cv2.split(ycrcb_watermarked)
    
    # Calculate the absolute difference between the Y channels
    y_difference = cv2.absdiff(y_original, y_watermarked)

    return y_difference

In [14]:
y_diff = compare_y_channels(original_image, watermarked_image)

In [15]:
import matplotlib.pyplot as plt

def visualize_y_diff(y_diff):
    plt.imshow(y_diff, cmap='gray')
    plt.title("Y Channel Differences")
    plt.show()

# Visualize the Y differences
# visualize_y_diff(y_diff)

Extracts the QR code based on the differences in the Y channels.

In [16]:
def extract_qr_from_dct_with_diff(dct_blocks, y_diff, threshold=0.00001):
    
    height, width = dct_blocks.shape
    qr_extracted = np.zeros((height // 8, width // 8), dtype=np.uint8)

    for i in range(0, height, 8):
        for j in range(0, width, 8):
            # print(f"Checking block ({i}, {j}) with Y-diff value: {y_diff[i, j]}")
            if y_diff[i, j] > threshold:
                qr_bit = 1 if dct_blocks[i:i+8, j:j+8][6, 6] > 0 else 0
                qr_extracted[i // 8, j // 8] = qr_bit
                # print(f"Extracting QR bit at block ({i // 8}, {j // 8}): {qr_bit}")
    
    # Convert the binary QR data into an image
    qr_image = Image.fromarray((qr_extracted * 255).astype(np.uint8))
    qr_image.save("extracted_qr_code.png")
    return qr_image

In [17]:
extracted_qr_code = extract_qr_from_dct_with_diff(dct_blocks, y_diff)

Verifies and decodes each QR tile from the extracted QR image.

In [18]:
def verify_qr_tiles(extracted_qr_image, qr_code_size=256):
    decoded_data = []
    width, height = extracted_qr_image.size
    for i in range(0, height, qr_code_size):
        for j in range(0, width, qr_code_size):
            qr_tile = extracted_qr_image.crop((j, i, j + qr_code_size, i + qr_code_size))
            decoded = decode(qr_tile)
            if decoded:
                decoded_data.append(decoded[0].data.decode('utf-8'))
    return decoded_data

In [19]:
decoded_qr_data = verify_qr_tiles(extracted_qr_code)
print(decoded_qr_data)

['https://example.com', 'https://example.com', 'https://example.com', 'https://example.com']
